In [ ]:
#@title Mount google drive to colab
#@markdown <br><center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height="50" alt="Gdrive-logo' "/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>
import sys
import requests
import pathlib
from google.colab import output, drive

Update_Error = [
"Unable to check for updates!",
"MCT Tool is out-dated! please update it to latest version to get new features or bug fixes."
]

Path_Error = [
"Custom Mount Path can't be empty! otherwise disable CUSTOM_MOUNT_PATH",
"Given Custom Mount Path should be Directory not a particular file.",
"Given Custom Mount Path not exist!"
]

update_from = "https://www.caduceus.ml/files/MCT-Version.txt"
current_version = "v4.4.8"

try:
  latest_version = requests.get(update_from).text
  f = 1
except:
  print(Update_Error[0])
  f = 0

if f == 1:
  if current_version == latest_version or latest_version == f"{current_version}\n":
    print(f"MCT Tool is updated! {current_version}")
  else:
    sys.exit(Update_Error[1])
else:
  pass

MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
FORCE_REMOUNT = True #@param {type:"boolean"}
CUSTOM_MOUNT_POINT = False #@param {type:"boolean"}
DEBUG_MODE = False #@param {type:"boolean"}

if CUSTOM_MOUNT_POINT == True:
  Custom_Mount_Path = input("Please enter Custom Mount Path:\n")
  if not Custom_Mount_Path:
    sys.exit(Path_Error[0])
  else:
    if pathlib.Path(Custom_Mount_Path).exists():
      if pathlib.Path(Custom_Mount_Path).is_dir():
        pass
      else:
        sys.exit(Path_Error[1])
    else:
      sys.exit(Path_Error[2])
else:
  Custom_Mount_Path = "/content/drive"

drive.mount._DEBUG = DEBUG_MODE
if MODE == "MOUNT":
  print("Please Wait! Mounting...")
  drive.mount(Custom_Mount_Path, force_remount= FORCE_REMOUNT)
  if DEBUG_MODE == False:
    output.clear()
  else:
    pass
  print(f"Mounted at {Custom_Mount_Path}")
elif MODE == "UNMOUNT":
  try:
    print("Please Wait! UNMOUNT in progress...")
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [ ]:
#@title To find the GPU being used
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16356199274184933608
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14415560704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5935489657076281256
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
#@title To install dependencies 
from google.colab import output
!pip install git+https://github.com/daniellerch/python-jpeg-toolbox 
!git clone https://github.com/daniellerch/hstego.git
!pip3 install imageio numpy scipy pycryptodome
!pip3 install git+https://github.com/daniellerch/hstego.git@v0.3
!pip install scikit-image
!python3 -m pip install SSIM-PIL
output.clear()
print("all the dependencies are installed")

all the dependencies are installed


In [ ]:
#@title convert colour image to grey image
path = "/content/drive/MyDrive/images/cover_jpeg/.jpg" #@param {type:"string"}
import cv2
from google.colab.patches import cv2_imshow
# Load the input image
image = cv2.imread(path)
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

filename = "G_10.jpg" #@param {type:"string"}
cv2.imwrite(filename, gray_image)

True

In [ ]:
#@title Crop the image to (512x512) {final cover pics}
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
path = "/content/drive/MyDrive/images/GREY_IMAGES/G_10.jpg" #@param {type:"string"}
image = cv2.imread(path)


y=0
x=0
h=512
w=512
crop = image[y:y+h, x:x+w]
filename = "cover_10.jpg" #@param {type:"string"}
cv2.imwrite(filename, crop)



In [ ]:
#@title Embed message in the image
#!/usr/bin/env python3

import os
import sys
import glob
import copy
import struct
import base64
import imageio
import hashlib
import hstegolib

import scipy.signal
import scipy.fftpack
import numpy as np

from ctypes import *
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

SPATIAL_EXT = ["png", "pgm", "tif"]
MAX_PAYLOAD=0.05
INF = 2**31-1



# {{{ is_ext()
def is_ext(path, extensions):
    fn, ext = os.path.splitext(path)
    if ext[1:].lower() in extensions:
        return True
    return False
# }}}

# {{{ jpeg_load()
def jpeg_load(path, use_blocks=False):

    if not os.path.isfile(path):
        raise FileNotFoundError(errno.ENOENT, 
                os.strerror(errno.ENOENT), path)

    jpeg.write_file.argtypes = c_char_p,
    jpeg.read_file.restype = py_object
    r = jpeg.read_file(path.encode())

    r["quant_tables"] = np.array(r["quant_tables"])

    for i in range(len(r["ac_huff_tables"])):
        r["ac_huff_tables"][i]["counts"] = np.array(r["ac_huff_tables"][i]["counts"])
        r["ac_huff_tables"][i]["symbols"] = np.array(r["ac_huff_tables"][i]["symbols"])

    for i in range(len(r["dc_huff_tables"])):
        r["dc_huff_tables"][i]["counts"] = np.array(r["dc_huff_tables"][i]["counts"])
        r["dc_huff_tables"][i]["symbols"] = np.array(r["dc_huff_tables"][i]["symbols"])

    if not use_blocks:
        chn = len(r["coef_arrays"])
        for c in range(chn):
            r["coef_arrays"][c] = np.array(r["coef_arrays"][c])
            h = r["coef_arrays"][c].shape[0]*8
            w = r["coef_arrays"][c].shape[1]*8
            r["coef_arrays"][c] = np.moveaxis(r["coef_arrays"][c], [0,1,2,3], [0,2,1,3])
            r["coef_arrays"][c] = r["coef_arrays"][c].reshape((h, w))

    return r
# }}}

# {{{ jpeg_save()
def jpeg_save(data, path, use_blocks=False):

    jpeg.write_file.argtypes = py_object,c_char_p

    r = copy.deepcopy(data)
    r["quant_tables"] = r["quant_tables"].astype('uint16').tolist()

    for i in range(len(r["ac_huff_tables"])):
        r["ac_huff_tables"][i]["counts"] = r["ac_huff_tables"][i]["counts"].tolist()
        r["ac_huff_tables"][i]["symbols"] = r["ac_huff_tables"][i]["symbols"].tolist()

    for i in range(len(r["dc_huff_tables"])):
        r["dc_huff_tables"][i]["counts"] = r["dc_huff_tables"][i]["counts"].tolist()
        r["dc_huff_tables"][i]["symbols"] = r["dc_huff_tables"][i]["symbols"].tolist()

    if not use_blocks:
        chn = len(r["coef_arrays"])
        for c in range(chn):
            h = r["coef_arrays"][c].shape[0]
            w = r["coef_arrays"][c].shape[1]
            r["coef_arrays"][c] = r["coef_arrays"][c].reshape((h//8, 8, w//8, 8))
            r["coef_arrays"][c] = np.moveaxis(r["coef_arrays"][c], [0,1,2,3], [0,2,1,3])
            r["coef_arrays"][c] = r["coef_arrays"][c].tolist()

    jpeg.write_file(r, path.encode())
# }}}

# {{{ jpg_capacity()
def jpg_capacity(jpg):
    """ channel capacity in bytes """
    total_capacity = 0
    for channel in range(len(jpg["coef_arrays"])):
        nz_coeff = np.count_nonzero(jpg["coef_arrays"][channel])
        capacity = int((nz_coeff*MAX_PAYLOAD)/8)
        f = capacity // 16
        capacity = (f-1)*16
        capacity -= 16+16+4 # data for header
        if capacity<0:
            capacity = 0
        total_capacity += capacity
    return total_capacity
# }}}

# {{{ spatial_capacity()
def spatial_capacity(I):
    m = 1
    for i in I.shape:
        m *= i

    capacity = int((m*MAX_PAYLOAD)/8)
    f = capacity // 16
    capacity = (f-1)*16
    capacity -= 16+16+4 # data for header
    if capacity<0:
        capacity = 0

    return capacity
# }}} 



class Cipher:
    # {{{
    def __init__(self, password):
        self.password = password

    def open_file(self, path):
        with open(path, 'rb') as f:
            self.plaintext = f.read()

    def aes_encrypt(self):
        salt = get_random_bytes(AES.block_size)

        # use the Scrypt KDF to get a private key from the password
        private_key = hashlib.scrypt(
            self.password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

        cipher = AES.new(private_key, AES.MODE_CBC)
        ciphertext = cipher.encrypt(pad(self.plaintext, AES.block_size))

        # Ciphertext with a 16+16 header 
        self.ciphertext = salt + cipher.iv + ciphertext

    def encrypt(self, input_path):
        """ encrypt file content """
        self.open_file(input_path)
        self.aes_encrypt()
        return self.ciphertext
  
    def aes_decrypt(self):
        salt = self.ciphertext[:AES.block_size]
        iv = self.ciphertext[AES.block_size:AES.block_size*2]
        ciphertext = self.ciphertext[AES.block_size*2:]

        # Fix padding
        mxlen = len(ciphertext)-(len(ciphertext)%AES.block_size)
        ciphertext = ciphertext[:mxlen]

        private_key = hashlib.scrypt(
            self.password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

        cipher = AES.new(private_key, AES.MODE_CBC, iv=iv)
        decrypted = cipher.decrypt(ciphertext)
        self.decrypted = unpad(decrypted, AES.block_size)

    def decrypt(self, bytes_array):
        """ decrypt """
        try:
            self.ciphertext = bytes_array
            self.aes_decrypt()
            return self.decrypted
        except:
            print("WARNING: message not found")
            return bytearray()
    # }}}

class Stego:
    # {{{
    def __init__(self):
        pass

    def bytes_to_bits(self, data):
        array=[]
        for b in data:
            for i in range(8):
                array.append((b >> i) & 1)
        return array


    def hide_stc(self, cover_array, costs_array, message_bits, mx=255, mn=0):

        cover = (c_int*(len(cover_array)))()
        for i in range(len(cover_array)):
            cover[i] = int(cover_array[i])

        # Prepare costs
        costs = (c_float*(len(costs_array)*3))()
        for i in range(len(costs_array)):
            if cover[i]<=mn:
                costs[3*i+0] = INF
                costs[3*i+1] = 0
                costs[3*i+2] = costs_array[i]
            elif cover[i]>=mx:
                costs[3*i+0] = costs_array[i]
                costs[3*i+1] = 0 
                costs[3*i+2] = INF
            else:
                costs[3*i+0] = costs_array[i]
                costs[3*i+1] = 0
                costs[3*i+2] = costs_array[i]


        m = len(message_bits)
        message = (c_ubyte*m)()
        for i in range(m):
            message[i] = message_bits[i]

        # Hide message
        stego = (c_int*(len(cover_array)))()
        _ = stc.stc_hide(len(cover_array), cover, costs, m, message, stego)

        # stego data to numpy
        stego_array = cover_array.copy()
        for i in range(len(cover_array)):
            stego_array[i] = stego[i]
     
        return stego_array


    def hide(self, message, cover_matrix, cost_matrix, mx=255, mn=0):
        message_bits = self.bytes_to_bits(message)

        height, width = cover_matrix.shape
        cover_array = cover_matrix.reshape((height*width,)) 
        costs_array = cost_matrix.reshape((height*width,)) 

        # Hide data_len (32 bits) into 64 pixels (0.5 payload)
        data_len = struct.pack("!I", len(message_bits))
        data_len_bits = self.bytes_to_bits(data_len)

        stego_array_1 = self.hide_stc(cover_array[:64], costs_array[:64], data_len_bits, mx, mn)
        stego_array_2 = self.hide_stc(cover_array[64:], costs_array[64:], message_bits, mx, mn)
        stego_array = np.hstack((stego_array_1, stego_array_2))

        stego_matrix = stego_array.reshape((height, width))
        
        return stego_matrix


    def unhide_stc(self, stego_array, message_len):

        stego = (c_int*(len(stego_array)))()
        for i in range(len(stego_array)):
            stego[i] = int(stego_array[i])

        extracted_message = (c_ubyte*len(stego_array))()
        s = stc.stc_unhide(len(stego_array), stego, message_len, extracted_message)

        if len(extracted_message) < message_len:
            print("WARNING, inconsistent message lenght:", 
                  len(extracted_message), ">", message_len)
            return bytearray()

        # Message bits to bytes
        data = bytearray()
        idx=0
        bitidx=0
        bitval=0
        for i in range(message_len):
            if bitidx==8:
                data.append(bitval)
                bitidx=0
                bitval=0
            bitval |= extracted_message[i]<<bitidx
            bitidx+=1
            idx += 1
        if bitidx==8:
            data.append(bitval)

        data = bytes(data)
        return data


    def unhide(self, stego_matrix):

        height, width = stego_matrix.shape
        stego_array = stego_matrix.reshape((height*width,))

        # Extract a 32-bits message lenght from a 64-pixel array
        data = self.unhide_stc(stego_array[:64], 32)
        data_len = struct.unpack_from("!I", data, 0)[0]
        
        data = self.unhide_stc(stego_array[64:], data_len)
        return data

    # }}}

class HILL:
    # {{{ 
    def cost_fn(self, I):                                                                
        HF1 = np.array([                                                             
            [-1, 2, -1],                                                             
            [ 2,-4,  2],                                                             
            [-1, 2, -1]                                                              
        ])                                                                           
        H2 = np.ones((3, 3)).astype(np.float)/3**2                                   
        HW = np.ones((15, 15)).astype(np.float)/15**2                                
                                                                                     
        R1 = scipy.signal.convolve2d(I, HF1, mode='same', boundary='symm')
        W1 = scipy.signal.convolve2d(np.abs(R1), H2, mode='same', boundary='symm')
        rho=1./(W1+10**(-10))
        cost = scipy.signal.convolve2d(rho, HW, mode='same', boundary='symm')

        cost[np.isnan(cost)] = INF
        cost[cost>INF] = INF

        return cost     


    def embed(self, input_img_path, msg_file_path, password, output_img_path):

        with open(msg_file_path, 'rb') as f:
            data = f.read()

        I = imageio.imread(input_img_path)
        
        n_channels = 3
        if len(I.shape) == 2:
            n_channels = 1
            I = I[..., np.newaxis]

        cipher = Cipher(password)
        message = cipher.encrypt(msg_file_path)

        # real capacity, without headers
        m = 1
        for i in I.shape:
            m *= i
        capacity = int((m*MAX_PAYLOAD)/8)
        if len(message) > capacity:
            print("ERROR, message too long:", len(message), ">", capacity)
            sys.exit(0)

        stego = Stego()

        if n_channels == 1:
            msg_bits = [ message ] 
        else:
            l = len(data)//3
            msg_bits = [ message[:l], message[l:2*l], message[2*l:] ]

        for c in range(n_channels):
            I[:,:,c] = stego.hide(msg_bits[c], I[:,:,c], self.cost_fn(I[:,:,c]))


        imageio.imwrite(output_img_path, I)


    def extract(self, stego_img_path, password, output_msg_path):

        I = imageio.imread(stego_img_path)
       
        n_channels = 3
        if len(I.shape) == 2:
            n_channels = 1
            I = I[..., np.newaxis]

        cipher = Cipher(password)
        stego = Stego()

        ciphertext = []
        for c in range(n_channels):
            ciphertext += stego.unhide(I[:,:,c])

        plain = cipher.decrypt(bytes(ciphertext))
        with open(output_msg_path, 'wb') as f:
            f.write(plain)

    # }}}

class J_UNIWARD:
    # {{{

    def dct2(self, a):
        return scipy.fftpack.dct(scipy.fftpack.dct(
                               a, axis=0, norm='ortho' ), axis=1, norm='ortho')
        
    def idct2(self, a):
        return scipy.fftpack.idct(scipy.fftpack.idct( 
                              a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

    def cost_fn(self, coef_arrays, quant_tables, spatial):

        hpdf = np.array([
            -0.0544158422,  0.3128715909, -0.6756307363,  0.5853546837,  
             0.0158291053, -0.2840155430, -0.0004724846,  0.1287474266,  
             0.0173693010, -0.0440882539, -0.0139810279,  0.0087460940,  
             0.0048703530, -0.0003917404, -0.0006754494, -0.0001174768
        ])        

        sign = np.array([-1 if i%2 else 1 for i in range(len(hpdf))])
        lpdf = hpdf[::-1] * sign

        F = []
        F.append(np.outer(lpdf.T, hpdf))
        F.append(np.outer(hpdf.T, lpdf))
        F.append(np.outer(hpdf.T, hpdf))


        # Pre-compute impact in spatial domain when a jpeg coefficient is changed by 1
        spatial_impact = {}
        for i in range(8):
            for j in range(8):
                test_coeffs = np.zeros((8, 8))
                test_coeffs[i, j] = 1
                spatial_impact[i, j] = self.idct2(test_coeffs) * quant_tables[i, j]

        # Pre compute impact on wavelet coefficients when a jpeg coefficient is changed by 1
        wavelet_impact = {}
        for f_index in range(len(F)):
            for i in range(8):
                for j in range(8):
                    wavelet_impact[f_index, i, j] = scipy.signal.correlate2d(spatial_impact[i, j], F[f_index], mode='full', boundary='fill', fillvalue=0.) # XXX


        # Create reference cover wavelet coefficients (LH, HL, HH)
        pad_size = 16 # XXX
        spatial_padded = np.pad(spatial, (pad_size, pad_size), 'symmetric')


        RC = []
        for i in range(len(F)):
            f = scipy.signal.correlate2d(spatial_padded, F[i], mode='same', boundary='fill')
            RC.append(f)

        coeffs = coef_arrays
        k, l = coeffs.shape
        nzAC = np.count_nonzero(coef_arrays) - np.count_nonzero(coef_arrays[::8, ::8])

        rho = np.zeros((k, l))
        tempXi = [0.]*3
        sgm = 2**(-6)

        # Computation of costs
        for row in range(k):
            for col in range(l):
                mod_row = row % 8
                mod_col = col % 8
                sub_rows = list(range(row-mod_row-6+pad_size-1, row-mod_row+16+pad_size))
                sub_cols = list(range(col-mod_col-6+pad_size-1, col-mod_col+16+pad_size))

                for f_index in range(3):
                    RC_sub = RC[f_index][sub_rows][:,sub_cols]
                    wav_cover_stego_diff = wavelet_impact[f_index, mod_row, mod_col]
                    tempXi[f_index] = abs(wav_cover_stego_diff) / (abs(RC_sub)+sgm)

                rho_temp = tempXi[0] + tempXi[1] + tempXi[2]
                rho[row, col] = np.sum(rho_temp)


        rho[np.isnan(rho)] = INF
        rho[rho>INF] = INF

        return rho

    def embed(self, input_img_path, msg_file_path, password, output_img_path):

        with open(msg_file_path, 'rb') as f:
            data = f.read()

        I = imageio.imread(input_img_path)
        jpg = jpeg_load(input_img_path)

        n_channels = 3
        if len(I.shape) == 2:
            n_channels = 1
            I = I[..., np.newaxis]

        cipher = Cipher(password)
        message = cipher.encrypt(msg_file_path)

        # Real capacity, without headers
        capacity = 0
        for channel in range(len(jpg["coef_arrays"])):
            nz_coeff = np.count_nonzero(jpg["coef_arrays"][channel])
            capacity += int((nz_coeff*MAX_PAYLOAD)/8)

        if len(message) > capacity:
            print("ERROR, message too long:", len(message), ">", capacity)
            sys.exit(0)


        stego = Stego()

        if n_channels == 1:
            msg_bits = [ message ] 
        else:
            l = len(data)//3
            msg_bits = [ message[:l], message[l:2*l], message[2*l:] ]

        for c in range(n_channels):
            quant = jpg["quant_tables"][0]
            if c > 2:
                quant = jpg["quant_tables"][1]

            cost = self.cost_fn(jpg["coef_arrays"][c], quant, I[:,:,c])
            jpg["coef_arrays"][c] = stego.hide(msg_bits[c], jpg["coef_arrays"][c], 
                                               cost, mx=1016, mn=-1016)

        jpeg_save(jpg, output_img_path)


    def extract(self, stego_img_path, password, output_msg_path):

        I = imageio.imread(stego_img_path)
        jpg = jpeg_load(stego_img_path)
       
        n_channels = 3
        if len(I.shape) == 2:
            n_channels = 1
            I = I[..., np.newaxis]

        cipher = Cipher(password)
        stego = Stego()

        ciphertext = []
        for c in range(n_channels):
            ciphertext += stego.unhide(jpg["coef_arrays"][c])

        plain = cipher.decrypt(bytes(ciphertext))

        f = open(output_msg_path, 'wb')
        f.write(plain)
        f.close()
        
    # }}}
input_img_path = "/content/cover_1.jpg" #@param {type:"string"}
msg_file_path = "/content/message.txt" #@param {type:"string"}
password = "isslab" #@param {type:"string"}
output_img_path= "/content/stego.jpg" #@param {type:"string"}

juniw = hstegolib.J_UNIWARD()

juniw.embed(input_img_path, msg_file_path, password, output_img_path)




In [ ]:
#@title Extract the message from the image
#!/usr/bin/env python3

import os
import sys
import glob
import copy
import struct
import base64
import imageio
import hashlib
import hstegolib

import scipy.signal
import scipy.fftpack
import numpy as np

from ctypes import *
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

SPATIAL_EXT = ["png", "pgm", "tif"]
MAX_PAYLOAD=0.05
INF = 2**31-1



# {{{ is_ext()
def is_ext(path, extensions):
    fn, ext = os.path.splitext(path)
    if ext[1:].lower() in extensions:
        return True
    return False
# }}}

# {{{ jpeg_load()
def jpeg_load(path, use_blocks=False):

    if not os.path.isfile(path):
        raise FileNotFoundError(errno.ENOENT, 
                os.strerror(errno.ENOENT), path)

    jpeg.write_file.argtypes = c_char_p,
    jpeg.read_file.restype = py_object
    r = jpeg.read_file(path.encode())

    r["quant_tables"] = np.array(r["quant_tables"])

    for i in range(len(r["ac_huff_tables"])):
        r["ac_huff_tables"][i]["counts"] = np.array(r["ac_huff_tables"][i]["counts"])
        r["ac_huff_tables"][i]["symbols"] = np.array(r["ac_huff_tables"][i]["symbols"])

    for i in range(len(r["dc_huff_tables"])):
        r["dc_huff_tables"][i]["counts"] = np.array(r["dc_huff_tables"][i]["counts"])
        r["dc_huff_tables"][i]["symbols"] = np.array(r["dc_huff_tables"][i]["symbols"])

    if not use_blocks:
        chn = len(r["coef_arrays"])
        for c in range(chn):
            r["coef_arrays"][c] = np.array(r["coef_arrays"][c])
            h = r["coef_arrays"][c].shape[0]*8
            w = r["coef_arrays"][c].shape[1]*8
            r["coef_arrays"][c] = np.moveaxis(r["coef_arrays"][c], [0,1,2,3], [0,2,1,3])
            r["coef_arrays"][c] = r["coef_arrays"][c].reshape((h, w))

    return r
# }}}

# {{{ jpeg_save()
def jpeg_save(data, path, use_blocks=False):

    jpeg.write_file.argtypes = py_object,c_char_p

    r = copy.deepcopy(data)
    r["quant_tables"] = r["quant_tables"].astype('uint16').tolist()

    for i in range(len(r["ac_huff_tables"])):
        r["ac_huff_tables"][i]["counts"] = r["ac_huff_tables"][i]["counts"].tolist()
        r["ac_huff_tables"][i]["symbols"] = r["ac_huff_tables"][i]["symbols"].tolist()

    for i in range(len(r["dc_huff_tables"])):
        r["dc_huff_tables"][i]["counts"] = r["dc_huff_tables"][i]["counts"].tolist()
        r["dc_huff_tables"][i]["symbols"] = r["dc_huff_tables"][i]["symbols"].tolist()

    if not use_blocks:
        chn = len(r["coef_arrays"])
        for c in range(chn):
            h = r["coef_arrays"][c].shape[0]
            w = r["coef_arrays"][c].shape[1]
            r["coef_arrays"][c] = r["coef_arrays"][c].reshape((h//8, 8, w//8, 8))
            r["coef_arrays"][c] = np.moveaxis(r["coef_arrays"][c], [0,1,2,3], [0,2,1,3])
            r["coef_arrays"][c] = r["coef_arrays"][c].tolist()

    jpeg.write_file(r, path.encode())
# }}}

# {{{ jpg_capacity()
def jpg_capacity(jpg):
    """ channel capacity in bytes """
    total_capacity = 0
    for channel in range(len(jpg["coef_arrays"])):
        nz_coeff = np.count_nonzero(jpg["coef_arrays"][channel])
        capacity = int((nz_coeff*MAX_PAYLOAD)/8)
        f = capacity // 16
        capacity = (f-1)*16
        capacity -= 16+16+4 # data for header
        if capacity<0:
            capacity = 0
        total_capacity += capacity
    return total_capacity
# }}}

# {{{ spatial_capacity()
def spatial_capacity(I):
    m = 1
    for i in I.shape:
        m *= i

    capacity = int((m*MAX_PAYLOAD)/8)
    f = capacity // 16
    capacity = (f-1)*16
    capacity -= 16+16+4 # data for header
    if capacity<0:
        capacity = 0

    return capacity
# }}} 



class Cipher:
    # {{{
    def __init__(self, password):
        self.password = password

    def open_file(self, path):
        with open(path, 'rb') as f:
            self.plaintext = f.read()

    def aes_encrypt(self):
        salt = get_random_bytes(AES.block_size)

        # use the Scrypt KDF to get a private key from the password
        private_key = hashlib.scrypt(
            self.password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

        cipher = AES.new(private_key, AES.MODE_CBC)
        ciphertext = cipher.encrypt(pad(self.plaintext, AES.block_size))

        # Ciphertext with a 16+16 header 
        self.ciphertext = salt + cipher.iv + ciphertext

    def encrypt(self, input_path):
        """ encrypt file content """
        self.open_file(input_path)
        self.aes_encrypt()
        return self.ciphertext
  
    def aes_decrypt(self):
        salt = self.ciphertext[:AES.block_size]
        iv = self.ciphertext[AES.block_size:AES.block_size*2]
        ciphertext = self.ciphertext[AES.block_size*2:]

        # Fix padding
        mxlen = len(ciphertext)-(len(ciphertext)%AES.block_size)
        ciphertext = ciphertext[:mxlen]

        private_key = hashlib.scrypt(
            self.password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

        cipher = AES.new(private_key, AES.MODE_CBC, iv=iv)
        decrypted = cipher.decrypt(ciphertext)
        self.decrypted = unpad(decrypted, AES.block_size)

    def decrypt(self, bytes_array):
        """ decrypt """
        try:
            self.ciphertext = bytes_array
            self.aes_decrypt()
            return self.decrypted
        except:
            print("WARNING: message not found")
            return bytearray()
    # }}}

class Stego:
    # {{{
    def __init__(self):
        pass

    def bytes_to_bits(self, data):
        array=[]
        for b in data:
            for i in range(8):
                array.append((b >> i) & 1)
        return array


    def hide_stc(self, cover_array, costs_array, message_bits, mx=255, mn=0):

        cover = (c_int*(len(cover_array)))()
        for i in range(len(cover_array)):
            cover[i] = int(cover_array[i])

        # Prepare costs
        costs = (c_float*(len(costs_array)*3))()
        for i in range(len(costs_array)):
            if cover[i]<=mn:
                costs[3*i+0] = INF
                costs[3*i+1] = 0
                costs[3*i+2] = costs_array[i]
            elif cover[i]>=mx:
                costs[3*i+0] = costs_array[i]
                costs[3*i+1] = 0 
                costs[3*i+2] = INF
            else:
                costs[3*i+0] = costs_array[i]
                costs[3*i+1] = 0
                costs[3*i+2] = costs_array[i]


        m = len(message_bits)
        message = (c_ubyte*m)()
        for i in range(m):
            message[i] = message_bits[i]

        # Hide message
        stego = (c_int*(len(cover_array)))()
        _ = stc.stc_hide(len(cover_array), cover, costs, m, message, stego)

        # stego data to numpy
        stego_array = cover_array.copy()
        for i in range(len(cover_array)):
            stego_array[i] = stego[i]
     
        return stego_array


    def hide(self, message, cover_matrix, cost_matrix, mx=255, mn=0):
        message_bits = self.bytes_to_bits(message)

        height, width = cover_matrix.shape
        cover_array = cover_matrix.reshape((height*width,)) 
        costs_array = cost_matrix.reshape((height*width,)) 

        # Hide data_len (32 bits) into 64 pixels (0.5 payload)
        data_len = struct.pack("!I", len(message_bits))
        data_len_bits = self.bytes_to_bits(data_len)

        stego_array_1 = self.hide_stc(cover_array[:64], costs_array[:64], data_len_bits, mx, mn)
        stego_array_2 = self.hide_stc(cover_array[64:], costs_array[64:], message_bits, mx, mn)
        stego_array = np.hstack((stego_array_1, stego_array_2))

        stego_matrix = stego_array.reshape((height, width))
        
        return stego_matrix


    def unhide_stc(self, stego_array, message_len):

        stego = (c_int*(len(stego_array)))()
        for i in range(len(stego_array)):
            stego[i] = int(stego_array[i])

        extracted_message = (c_ubyte*len(stego_array))()
        s = stc.stc_unhide(len(stego_array), stego, message_len, extracted_message)

        if len(extracted_message) < message_len:
            print("WARNING, inconsistent message lenght:", 
                  len(extracted_message), ">", message_len)
            return bytearray()

        # Message bits to bytes
        data = bytearray()
        idx=0
        bitidx=0
        bitval=0
        for i in range(message_len):
            if bitidx==8:
                data.append(bitval)
                bitidx=0
                bitval=0
            bitval |= extracted_message[i]<<bitidx
            bitidx+=1
            idx += 1
        if bitidx==8:
            data.append(bitval)

        data = bytes(data)
        return data


    def unhide(self, stego_matrix):

        height, width = stego_matrix.shape
        stego_array = stego_matrix.reshape((height*width,))

        # Extract a 32-bits message lenght from a 64-pixel array
        data = self.unhide_stc(stego_array[:64], 32)
        data_len = struct.unpack_from("!I", data, 0)[0]
        
        data = self.unhide_stc(stego_array[64:], data_len)
        return data

    # }}}

class HILL:
    # {{{ 
    def cost_fn(self, I):                                                                
        HF1 = np.array([                                                             
            [-1, 2, -1],                                                             
            [ 2,-4,  2],                                                             
            [-1, 2, -1]                                                              
        ])                                                                           
        H2 = np.ones((3, 3)).astype(np.float)/3**2                                   
        HW = np.ones((15, 15)).astype(np.float)/15**2                                
                                                                                     
        R1 = scipy.signal.convolve2d(I, HF1, mode='same', boundary='symm')
        W1 = scipy.signal.convolve2d(np.abs(R1), H2, mode='same', boundary='symm')
        rho=1./(W1+10**(-10))
        cost = scipy.signal.convolve2d(rho, HW, mode='same', boundary='symm')

        cost[np.isnan(cost)] = INF
        cost[cost>INF] = INF

        return cost     


    def embed(self, input_img_path, msg_file_path, password, output_img_path):

        with open(msg_file_path, 'rb') as f:
            data = f.read()

        I = imageio.imread(input_img_path)
        
        n_channels = 3
        if len(I.shape) == 2:
            n_channels = 1
            I = I[..., np.newaxis]

        cipher = Cipher(password)
        message = cipher.encrypt(msg_file_path)

        # real capacity, without headers
        m = 1
        for i in I.shape:
            m *= i
        capacity = int((m*MAX_PAYLOAD)/8)
        if len(message) > capacity:
            print("ERROR, message too long:", len(message), ">", capacity)
            sys.exit(0)

        stego = Stego()

        if n_channels == 1:
            msg_bits = [ message ] 
        else:
            l = len(data)//3
            msg_bits = [ message[:l], message[l:2*l], message[2*l:] ]

        for c in range(n_channels):
            I[:,:,c] = stego.hide(msg_bits[c], I[:,:,c], self.cost_fn(I[:,:,c]))


        imageio.imwrite(output_img_path, I)


    def extract(self, stego_img_path, password, output_msg_path):

        I = imageio.imread(stego_img_path)
       
        n_channels = 3
        if len(I.shape) == 2:
            n_channels = 1
            I = I[..., np.newaxis]

        cipher = Cipher(password)
        stego = Stego()

        ciphertext = []
        for c in range(n_channels):
            ciphertext += stego.unhide(I[:,:,c])

        plain = cipher.decrypt(bytes(ciphertext))
        with open(output_msg_path, 'wb') as f:
            f.write(plain)

    # }}}

class J_UNIWARD:
    # {{{

    def dct2(self, a):
        return scipy.fftpack.dct(scipy.fftpack.dct(
                               a, axis=0, norm='ortho' ), axis=1, norm='ortho')
        
    def idct2(self, a):
        return scipy.fftpack.idct(scipy.fftpack.idct( 
                              a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

    def cost_fn(self, coef_arrays, quant_tables, spatial):

        hpdf = np.array([
            -0.0544158422,  0.3128715909, -0.6756307363,  0.5853546837,  
             0.0158291053, -0.2840155430, -0.0004724846,  0.1287474266,  
             0.0173693010, -0.0440882539, -0.0139810279,  0.0087460940,  
             0.0048703530, -0.0003917404, -0.0006754494, -0.0001174768
        ])        

        sign = np.array([-1 if i%2 else 1 for i in range(len(hpdf))])
        lpdf = hpdf[::-1] * sign

        F = []
        F.append(np.outer(lpdf.T, hpdf))
        F.append(np.outer(hpdf.T, lpdf))
        F.append(np.outer(hpdf.T, hpdf))


        # Pre-compute impact in spatial domain when a jpeg coefficient is changed by 1
        spatial_impact = {}
        for i in range(8):
            for j in range(8):
                test_coeffs = np.zeros((8, 8))
                test_coeffs[i, j] = 1
                spatial_impact[i, j] = self.idct2(test_coeffs) * quant_tables[i, j]

        # Pre compute impact on wavelet coefficients when a jpeg coefficient is changed by 1
        wavelet_impact = {}
        for f_index in range(len(F)):
            for i in range(8):
                for j in range(8):
                    wavelet_impact[f_index, i, j] = scipy.signal.correlate2d(spatial_impact[i, j], F[f_index], mode='full', boundary='fill', fillvalue=0.) # XXX


        # Create reference cover wavelet coefficients (LH, HL, HH)
        pad_size = 16 # XXX
        spatial_padded = np.pad(spatial, (pad_size, pad_size), 'symmetric')


        RC = []
        for i in range(len(F)):
            f = scipy.signal.correlate2d(spatial_padded, F[i], mode='same', boundary='fill')
            RC.append(f)

        coeffs = coef_arrays
        k, l = coeffs.shape
        nzAC = np.count_nonzero(coef_arrays) - np.count_nonzero(coef_arrays[::8, ::8])

        rho = np.zeros((k, l))
        tempXi = [0.]*3
        sgm = 2**(-6)

        # Computation of costs
        for row in range(k):
            for col in range(l):
                mod_row = row % 8
                mod_col = col % 8
                sub_rows = list(range(row-mod_row-6+pad_size-1, row-mod_row+16+pad_size))
                sub_cols = list(range(col-mod_col-6+pad_size-1, col-mod_col+16+pad_size))

                for f_index in range(3):
                    RC_sub = RC[f_index][sub_rows][:,sub_cols]
                    wav_cover_stego_diff = wavelet_impact[f_index, mod_row, mod_col]
                    tempXi[f_index] = abs(wav_cover_stego_diff) / (abs(RC_sub)+sgm)

                rho_temp = tempXi[0] + tempXi[1] + tempXi[2]
                rho[row, col] = np.sum(rho_temp)


        rho[np.isnan(rho)] = INF
        rho[rho>INF] = INF

        return rho

    def embed(self, input_img_path, msg_file_path, password, output_img_path):

        with open(msg_file_path, 'rb') as f:
            data = f.read()

        I = imageio.imread(input_img_path)
        jpg = jpeg_load(input_img_path)

        n_channels = 3
        if len(I.shape) == 2:
            n_channels = 1
            I = I[..., np.newaxis]

        cipher = Cipher(password)
        message = cipher.encrypt(msg_file_path)

        # Real capacity, without headers
        capacity = 0
        for channel in range(len(jpg["coef_arrays"])):
            nz_coeff = np.count_nonzero(jpg["coef_arrays"][channel])
            capacity += int((nz_coeff*MAX_PAYLOAD)/8)

        if len(message) > capacity:
            print("ERROR, message too long:", len(message), ">", capacity)
            sys.exit(0)


        stego = Stego()

        if n_channels == 1:
            msg_bits = [ message ] 
        else:
            l = len(data)//3
            msg_bits = [ message[:l], message[l:2*l], message[2*l:] ]

        for c in range(n_channels):
            quant = jpg["quant_tables"][0]
            if c > 2:
                quant = jpg["quant_tables"][1]

            cost = self.cost_fn(jpg["coef_arrays"][c], quant, I[:,:,c])
            jpg["coef_arrays"][c] = stego.hide(msg_bits[c], jpg["coef_arrays"][c], 
                                               cost, mx=1016, mn=-1016)

        jpeg_save(jpg, output_img_path)


    def extract(self, stego_img_path, password, output_msg_path):

        I = imageio.imread(stego_img_path)
        jpg = jpeg_load(stego_img_path)
       
        n_channels = 3
        if len(I.shape) == 2:
            n_channels = 1
            I = I[..., np.newaxis]

        cipher = Cipher(password)
        stego = Stego()

        ciphertext = []
        for c in range(n_channels):
            ciphertext += stego.unhide(jpg["coef_arrays"][c])

        plain = cipher.decrypt(bytes(ciphertext))

        f = open(output_msg_path, 'wb')
        f.write(plain)
        f.close()
        
    # }}}
stego_img_path = "/content/stego.jpg" #@param {type:"string"}
password = "isslab" #@param {type:"string"}
output_msg_path = "decrypt.txt" #@param {type:"string"}

juniw = hstegolib.J_UNIWARD()
juniw.extract(stego_img_path, password, output_msg_path)




In [ ]:
#@title Compare the similarity between cover and stego images using SSIM(The structural similarity index measure)
from SSIM_PIL import compare_ssim
from PIL import Image
path1 = "/content/cover_1.jpg" #@param {type:"string"}
path2 = "/content/stego.jpg" #@param {type:"string"}
image1 = Image.open(path1)
image2 = Image.open(path2)

# Compare images using OpenCL by default
value = compare_ssim(image1, image2)
print(value)

#  Compare images using GPU-only version
value = compare_ssim(image1, image2, GPU=True)
print(value)


No module named 'pyopencl'
0.9999780967686853
No module named 'pyopencl'
0.9999780967686853
